# Deploy and perform inference on Model Package from AWS Marketplace 

This notebook provides you instructions on how to deploy and perform inference on model packages from AWS Marketplace Sentence Pair Classification model.

This notebook is compatible only with those Sentence Pair Classification model packages which this notebook is linked to.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to this Sentence Pair Classification model. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Delete the endpoint](#D.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

**Note** - This notebook requires you to follow instructions and specify values for parameters, as instructed.

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page you opened this notebook for.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn='<Customer to specify Model package ARN corresponding to their AWS region>' 

In [ ]:
import json 
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()
boto3 = sagemaker_session.boto_session
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name

s3 = boto3.client("s3")
runtime= boto3.client('runtime.sagemaker')

In next step, you would be deploying the model for real-time inference. For  information on how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### 2. Create an endpoint and perform real-time inference

In [ ]:
model_name='sentence-pair-classification-model'

In [ ]:
#The Sentence Pair Classification model packages this notebook notebook is compatible with, support application/list-text as the 
#content-type.
content_type='application/list-text'

Review and update the compatible instance type for the model package in the following cell.

In [ ]:
real_time_inference_instance_type='ml.g4dn.xlarge'
batch_transform_inference_instance_type='ml.p2.xlarge'

#### A. Create an endpoint

In [ ]:
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Prepare input file for performing real-time inference
#### Let's put in some example sentence pairs. You can put in any pairs of sentences, the model will predict whether the second sentence entails the first sentence or not.
These examples are taken from QNLI dataset downloaded from [TensorFlow](https://www.tensorflow.org/datasets/catalog/glue#glueqnli). [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). [Dataset Homepage](https://rajpurkar.github.io/SQuAD-explorer/). [CC BY-SA 4.0 License](https://creativecommons.org/licenses/by-sa/4.0/legalcode). Citations:

<sub><sup>
@article{rajpurkar2016squad,
  title={Squad: 100,000+ questions for machine comprehension of text},
  author={Rajpurkar, Pranav and Zhang, Jian and Lopyrev, Konstantin and Liang, Percy},
  journal={arXiv preprint arXiv:1606.05250},
  year={2016}
}
@inproceedings{wang2019glue,
  title={ {GLUE}: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding},
  author={Wang, Alex and Singh, Amanpreet and Michael, Julian and Hill, Felix and Levy, Omer and Bowman, Samuel R.},
  note={In the Proceedings of ICLR.},
  year={2019}
}
</sup></sub>

In [ ]:
sentence_pair1 = ["How many octaves does Beyonce have?", "Beyoncé's vocal range spans four octaves."]
sentence_pair2 = ["How many octaves does Beyonce have?", "While another critic says she is a ""Vocal acrobat, being able to sing long and complex melismas and vocal runs effortlessly, and in key."]

#### C. Query endpoint that you have created

In [ ]:
#perform_inference method performs inference on the endpoint and prints predictions.
label_map = {0: "entailment", 1: "no-entailment"}
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
def query_endpoint(encoded_text):
    endpoint_name = model_name
    response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType=content_type, Body=encoded_text)
    model_predictions = json.loads(response['Body'].read())[0]
    return model_predictions

for sentence_pair in [sentence_pair1, sentence_pair2]:
    model_predictions = query_endpoint(json.dumps(sentence_pair).encode('utf-8'))
    class_index = model_predictions.index(max(model_predictions))
    print (f"Inference:{newline}"
            f"first-sentence: {sentence_pair[0]}{newline}"
            f"second-sentence: {sentence_pair[1]}{newline}"
            f"model prediction: {model_predictions}{newline}"
            f"model prediction: {bold}{label_map[class_index]}{unbold}{newline}")

#### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
#upload the batch-transform job input files to S3
transform_input_key_prefix = 'sentence-pair-classification-model-transform-input'
f = open("transform-input-data.txt", "w")
f.write("[\"How many octaves does Beyonce have?\", \"Beyoncé's vocal range spans four octaves.\"]")
f.close()
transform_input = sagemaker_session.upload_data("transform-input-data.txt", key_prefix=transform_input_key_prefix) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

In [ ]:
output_bucket_name, output_path = transformer.output_path.replace("s3://", "").split("/", 1)
obj = s3.get_object(Bucket=output_bucket_name, Key=output_path + '/transform-input-data.txt.out')
batch_prediction = obj['Body'].read().decode('utf-8')

# print out batch-transform job output
print(batch_prediction)

### 4. Clean-up

#### A. Delete the model

In [ ]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

